## Research Objective

**Hypotheses**: 

The hypothesis is that low-income countries might be less dependent on flights for tourism, possibly due to regional tourism or overland travel methods (like buses, trains, or cars)

**Main question**: 

How do flight fluctuations during the COVID-19 period (2019-2022) influence tourism across different income classes, and can clustering of countries based on various metrics reveal unique responses within these classes?
Eventually, with Kmeans, a clustering is done to check of there is indeed correlation

**Sub questions**:
- 1: Impact on Tourism by Flights: How has the decline in flight numbers directly impacted tourism figures across The Netherlands, France, Bulgaria, and other EU countries? *This gives an initial broad view of the relationship between flights and tourism across the board.*

- 2: Holistic Clustering for Patterns: *By considering metrics related to tourism, flight activity, GHG emissions, and supply chains, can we identify clusters that correspond to a country's income class or their reliance on flights for tourism?*

- 3: Inter-Class Analysis: *How do the impacts on tourism due to flight reductions vary between high, middle, and low-income countries? Within these classes, are there outliers or countries that defy the general pattern?*

